# Pre-processing Delaware Riber Basin Commission Aggregated Amounts data for WaDE upload.
Date Updated: 06/05/2023
Purpose:  To pre-process the DRBC data into one master file for simple DataFrame creation and extraction

In [1]:
#Needed Libararies

# working with data
import os
import numpy as np
import pandas as pd
import geopandas as gpd

# visulizaiton
import matplotlib.pyplot as plt
import seaborn as sns

# API retrieval
import requests
import json

# Cleanup
import re
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
pd.set_option('display.float_format', lambda x: '%.5f' % x) # suppress scientific notation in Pandas

In [2]:
# Working Directory
workingDir = "G:/Shared drives/WaDE Data/DelawareRiverBasinCommission/AggregatedAmounts/RawInputData"
os.chdir(workingDir)

## Input Source Data

In [ ]:
# timeseries data, xlsx file
# we only want the historic surface water and groundwater data. Which can be found on the following sheets: A-1, A-6, A-9, A-11, A-14, A-17, A-22
dfA1 = pd.read_excel(io="2060report_data-release_v2110.xlsx", sheet_name='A-1')
dfA6 = pd.read_excel(io="2060report_data-release_v2110.xlsx", sheet_name='A-6')
dfA9 = pd.read_excel(io="2060report_data-release_v2110.xlsx", sheet_name='A-9')
dfA11 = pd.read_excel(io="2060report_data-release_v2110.xlsx", sheet_name='A-11')
dfA14 = pd.read_excel(io="2060report_data-release_v2110.xlsx", sheet_name='A-14')
dfA17 = pd.read_excel(io="2060report_data-release_v2110.xlsx", sheet_name='A-17')
dfA22 = pd.read_excel(io="2060report_data-release_v2110.xlsx", sheet_name='A-22')

# Concatenate dataframes
frames = [dfA1, dfA6, dfA9, dfA11, dfA14, dfA17, dfA22] 
dfIn = pd.concat(frames)
dfIn = dfIn.drop_duplicates().reset_index(drop=True).replace(np.nan, "")


# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfIn:
    dfIn['WaDEUUID'] = "drbc" + dfIn.index.astype(str)
    dfIn.to_csv('2060report_data-release_v2110_A1_6_9_11_14_17_22.zip', compression=dict(method='zip', archive_name='2060report_data-release_v2110_A1_6_9_11_14_17_22.csv'), index=False)

print(len(dfIn))
dfIn.head(1)

In [ ]:
# shapefile info
gdf_drb147 = gpd.read_file('shapefile/drb147.shp', crs="EPSG:4326")
print(len(gdf_drb147))
gdf_drb147.head(1)

In [ ]:
# left merge timeseries info with shapefile info.
dfIn2 = pd.DataFrame()
dfIn2 = pd.merge(dfIn, gdf_drb147, on='BASIN_ID', how='left')
dfIn2 = dfIn2.drop_duplicates().replace(np.nan, "").replace("nan,nan", "").reset_index(drop=True)
print(len(dfIn2))
dfIn2.head(1)

## Convert to WaDE Input Data

In [ ]:
# covert MGPD to AFY to
# 1000000 US liquid gallons / Day = 1120.14406 Acre-foot / year.

def convertMGPDtoAFYFunc(Val):
    Val = str(Val).strip()
    if (Val == "" ) or (pd.isnull(Val)):
        outList = ""
    else:
        outList = float(Val) * 1120.14406 
    return outList

In [ ]:
# withdrawal values
dfIn2['in_WD_AFY'] = dfIn2.apply(lambda row: convertMGPDtoAFYFunc(row['WD_MGD']), axis=1)
dfIn2['in_WD_AFY'].unique()

In [ ]:
# consumptive values
dfIn2['in_CU_AFY'] = dfIn2.apply(lambda row: convertMGPDtoAFYFunc(row['CU_MGD']), axis=1)
dfIn2['in_CU_AFY'].unique()

In [ ]:
# Withdrawal dataframe
# --------------------------

# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfIn2['WaDEUUID']

# Method Info
df['in_MethodUUID'] = "DRBCag_M1"

# Variable Info
df['in_VariableSpecificUUID'] = "" # determine below
df['in_AggregationIntervalUnitCV'] = "Annual"
df['in_VariableCV'] = "Withdrawal"
df['in_VariableSpecificCV'] = "" # determine below

# Organization Info
df['in_OrganizationUUID'] = "DRBCag_O1"

# Water Source
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df["in_WaterSourceName"] = ""
df['in_WaterSourceNativeID'] = ""
df["in_WaterSourceTypeCV"] = dfIn2['DESIGNATION']

# ReportingUnits Info
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_ReportingUnitName'] = dfIn2['STREAMS']
df['in_ReportingUnitNativeID'] = dfIn2['BASIN_ID'].replace("", 0).fillna(0).astype(str).str.strip()
df['in_ReportingUnitProductVersion'] = ""
df['in_ReportingUnitTypeCV'] = "Subbasin"
df['in_ReportingUnitUpdateDate'] = ""
df['in_StateCV'] = "DE"

# AggregatedAmounts Info
df['in_AllocationCropDutyAmount'] = ""
df['in_Amount'] = dfIn2['in_WD_AFY'] #use withdrawl value here, see above for conversion
df['in_BeneficialUseCategory'] = dfIn2['CATEGORY']
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_InterbasinTransferFromID'] = ""
df['in_InterbasinTransferToID'] = ""
df['in_IrrigatedAcreage'] = ""
df['in_IrrigationMethodCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerGeneratedGWh'] = ""
df['in_PowerType'] = ""
df['in_PrimaryUseCategory'] = ""
df['in_ReportYearCV'] = dfIn2['YEAR']
df['in_SDWISIdentifierCV'] = ""
df['in_TimeframeEnd'] = "" # determine below
df['in_TimeframeStart'] = "" # determine below

outWith = df.copy()
outWith = outWith.drop_duplicates().reset_index(drop=True)
print(len(outWith))
outWith.head()

In [ ]:
# Consumptive dataframe
# --------------------------

# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfIn2['WaDEUUID']

# Method Info
df['in_MethodUUID'] = "DRBCag_M1"

# Variable Info
df['in_VariableSpecificUUID'] = "" # determine below
df['in_AggregationIntervalUnitCV'] = "Annual"
df['in_VariableCV'] = "Consumptive"
df['in_VariableSpecificCV'] = "" # determine below

# Organization Info
df['in_OrganizationUUID'] = "DRBCag_O1"

# Water Source
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df["in_WaterSourceName"] = ""
df['in_WaterSourceNativeID'] = ""
df["in_WaterSourceTypeCV"] = dfIn2['DESIGNATION']

# ReportingUnits Info
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_ReportingUnitName'] = dfIn2['STREAMS']
df['in_ReportingUnitNativeID'] = dfIn2['BASIN_ID'].replace("", 0).fillna(0).astype(str).str.strip()
df['in_ReportingUnitProductVersion'] = ""
df['in_ReportingUnitTypeCV'] = "Subbasin"
df['in_ReportingUnitUpdateDate'] = ""
df['in_StateCV'] = "DE"

# AggregatedAmounts Info
df['in_AllocationCropDutyAmount'] = ""
df['in_Amount'] = dfIn2['in_CU_AFY'] #use consumptive value here, see above for conversion
df['in_BeneficialUseCategory'] = dfIn2['CATEGORY']
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_InterbasinTransferFromID'] = ""
df['in_InterbasinTransferToID'] = ""
df['in_IrrigatedAcreage'] = ""
df['in_IrrigationMethodCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerGeneratedGWh'] = ""
df['in_PowerType'] = ""
df['in_PrimaryUseCategory'] = ""
df['in_ReportYearCV'] = dfIn2['YEAR']
df['in_SDWISIdentifierCV'] = ""
df['in_TimeframeEnd'] = "" # determine below
df['in_TimeframeStart'] = "" # determine below

outCons = df.copy()
outCons = outCons.drop_duplicates().reset_index(drop=True)
print(len(outCons))
outCons.head()

In [ ]:
# Concatenate dataframes together
frames = [outWith, outCons] #withdrawl, consumptive
outdf = pd.concat(frames)
outdf = outdf.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
print(len(outdf))

## WaDE Custom Elements (due to missing sate info)

In [ ]:
# determine water source type from provided abbreviations.

def determineWaterSourceTypeCVFunc(Val):
    Val = str(Val).strip()
    if (Val == "GW"):
        outString = "Groundwater"
    elif (Val == "SW"):
        outString = "Surface Water"
    else:
        outString = ""
    return outString

outdf['in_WaterSourceTypeCV'] = outdf.apply(lambda row: determineWaterSourceTypeCVFunc(row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceTypeCV'].unique()

In [ ]:
# create in_TimeframeEnd & in_TimeframeStart, use in_ReportYearCV as input.

def determineReportYearCVFunc(mdVal, Val):
    Val = str(int(Val)).strip()
    if (Val == "" ) or (pd.isnull(Val)):
        outString = ""
    else:
        outString = mdVal + Val
    return outString

In [ ]:
# in_TimeframeEnd values
outdf['in_TimeframeEnd'] = outdf.apply(lambda row: determineReportYearCVFunc("12/31/", row['in_ReportYearCV']), axis=1)
outdf['in_TimeframeEnd'].unique()

In [ ]:
# in_TimeframeStart values
outdf['in_TimeframeStart'] = outdf.apply(lambda row: determineReportYearCVFunc("01/01/", row['in_ReportYearCV']), axis=1)
outdf['in_TimeframeStart'].unique()

In [ ]:
# Clean owner name up
def removeSpecialCharsFunc(Val):
    Val = str(Val)
    Val = re.sub("[$@&.;,/\)(-]", "", Val).title().replace("  ", " ").strip()
    return Val

In [ ]:
outdf['in_ReportingUnitName'] = outdf.apply(lambda row: removeSpecialCharsFunc(row['in_ReportingUnitName']), axis=1)
outdf['in_ReportingUnitName'].unique()

In [ ]:
outdf['in_WaterSourceName'] = outdf.apply(lambda row: removeSpecialCharsFunc(row['in_WaterSourceName']), axis=1)
outdf['in_WaterSourceName'].unique()

In [ ]:
def ensureEmptyString(val):
    val = str(val).strip()
    if val == "" or val == " " or val == "nan" or pd.isnull(val):
        outString = ""
    else:
        outString = val
    return outString

In [ ]:
outdf['in_WaterSourceName'] = outdf.apply(lambda row: ensureEmptyString(row['in_WaterSourceName']), axis=1)
outdf['in_WaterSourceName'].unique()

In [ ]:
outdf['in_WaterSourceTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceTypeCV'].unique()

In [ ]:
outdf['in_ReportingUnitName'] = outdf.apply(lambda row: ensureEmptyString(row['in_ReportingUnitName']), axis=1)
outdf['in_ReportingUnitName'].unique()

In [ ]:
outdf['in_ReportingUnitTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_ReportingUnitTypeCV']), axis=1)
outdf['in_ReportingUnitTypeCV'].unique()

In [ ]:
outdf['in_BeneficialUseCategory'] = outdf.apply(lambda row: ensureEmptyString(row['in_BeneficialUseCategory']), axis=1)
for x in outdf['in_BeneficialUseCategory'].sort_values().unique():
    print(f'"' + x + '",')

In [ ]:
# Fixing n_Amount datatype
outdf['in_Amount'] = pd.to_numeric(outdf['in_Amount'], errors='coerce').replace(0,"").fillna("")
outdf['in_Amount'].unique()

In [ ]:
%%time

# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "wadeID" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceName'] = outdf['in_WaterSourceName']
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = outdf['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)
dfWaterSourceNativeID['linkKey'] = dfWaterSourceNativeID['in_WaterSourceName'].astype(str) + dfWaterSourceNativeID['in_WaterSourceTypeCV'].astype(str)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
WaterSourceNativeIDdict = pd.Series(dfWaterSourceNativeID.in_WaterSourceNativeID.values, index=dfWaterSourceNativeID.linkKey.astype(str)).to_dict()
def retrieveWaterSourceNativeID(A, B):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        colrowValue = str(A).strip() + str(B).strip()
        try:
            outList = WaterSourceNativeIDdict[colrowValue]
        except:
            outList = ''
    return outList

outdf['in_WaterSourceNativeID'] = outdf.apply(lambda row: retrieveWaterSourceNativeID( row['in_WaterSourceName'], row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceNativeID'].unique()

In [ ]:
# determine in_VariableSpecificCV
outdf['in_VariableSpecificCV'] = outdf['in_VariableCV'].astype(str) + "_" + outdf['in_AggregationIntervalUnitCV'].astype(str) + "_" + outdf['in_BeneficialUseCategory'].astype(str) + "_" + outdf['in_WaterSourceTypeCV'].astype(str)
outdf['in_VariableSpecificCV'].unique()

In [ ]:
%%time

# Creating WaDE VariableSpecificUUID for easy VariableSpecificCV identification 
# use these inputs: VariableCV_AggregationIntervalUnitCV_BeneficalUse_WaterSourceTypeCV
# ----------------------------------------------------------------------------------------------------

# Create temp VariableSpecificUUID dataframe of unique water source.
def assignVariableSpecificUUID(colrowValue):
    string1 = str(colrowValue)
    outstring = "DRBCwr_V" + string1
    return outstring

dfVariableSpecificUUID = pd.DataFrame()
dfVariableSpecificUUID['in_VariableCV'] = outdf['in_VariableCV']
dfVariableSpecificUUID['in_AggregationIntervalUnitCV'] = outdf['in_AggregationIntervalUnitCV']
dfVariableSpecificUUID['in_BeneficialUseCategory'] = outdf['in_BeneficialUseCategory']
dfVariableSpecificUUID['in_WaterSourceTypeCV'] = outdf['in_WaterSourceTypeCV']
dfVariableSpecificUUID = dfVariableSpecificUUID.drop_duplicates()

dftemp = pd.DataFrame(index=dfVariableSpecificUUID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfVariableSpecificUUID['in_VariableSpecificUUID'] = dftemp.apply(lambda row: assignVariableSpecificUUID(row['Count']), axis=1)
dfVariableSpecificUUID['linkKey'] = dfVariableSpecificUUID['in_VariableCV'].astype(str) + dfVariableSpecificUUID['in_AggregationIntervalUnitCV'].astype(str) + dfVariableSpecificUUID['in_BeneficialUseCategory'].astype(str) + dfVariableSpecificUUID['in_WaterSourceTypeCV'].astype(str)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
VariableSpecificUUIDdict = pd.Series(dfVariableSpecificUUID.in_VariableSpecificUUID.values, index=dfVariableSpecificUUID.linkKey.astype(str)).to_dict()
def retrieveVariableSpecificUUID(A, B, C, D):
    if (A == '' and B == '' and C == '' and D == '') or (pd.isnull(A) and pd.isnull(B) and pd.isnull(C) and pd.isnull(D)):
        outList = ''
    else:
        colrowValue = str(A).strip() + str(B).strip() + str(C).strip() + str(D).strip()
        try:
            outList = VariableSpecificUUIDdict[colrowValue]
        except:
            outList = ''
    return outList

outdf['in_VariableSpecificUUID'] = outdf.apply(lambda row: retrieveVariableSpecificUUID(row['in_VariableCV'], row['in_AggregationIntervalUnitCV'], row['in_BeneficialUseCategory'], row['in_WaterSourceTypeCV']), axis=1)
outdf['in_VariableSpecificUUID'].unique()

## Shapefile Data
- For attaching geometry to csv inputs.

In [ ]:
# PoU Shapefile Data
# Shapefile input
dfshapetemp = gpd.read_file('shapefile/drb147.shp', crs="EPSG:4326")
print(len(dfshapetemp))
dfshapetemp.head()

In [ ]:
columnsList = ['in_ReportingUnitNativeID', 'geometry']
outshape = pd.DataFrame(columns=columnsList)
outshape['in_ReportingUnitNativeID'] = "id" + dfshapetemp['BASIN_ID'].replace("", 0).fillna(0).astype(str).str.strip()
outshape['geometry'] = dfshapetemp['geometry']
outshape = outshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
outshape.head()

# Export the Output 

In [ ]:
outdf.info

In [ ]:
outdf

In [ ]:
# Export the output dataframe
outdf.to_csv('Pag_drbcMain.zip', compression=dict(method='zip', archive_name='Pag_drbcMain.csv'), index=False)   # The output, save as a zip
outshape.to_csv('P_Geometry.zip', compression=dict(method='zip', archive_name='P_Geometry.csv'), index=False)  # The output geometry.